In [1]:
from ase.formula import Formula

Formula("NaCl") * 5 // Formula("NaCl")

5

In [3]:
from muse.transforms.mixture import mix
from ase.io import write
from ase.calculators.lj import LennardJones
from ase.optimize import FIRE
from vasp_interactive import VaspInteractive
from ase.calculators.socketio import SocketIOCalculator
from ase.md.npt import NPT
from ase import units
import numpy as np

tolerance = 2.0

atoms = mix(
    recipe={"NaCl": 8, "UCl3": 10},
    tolerance=tolerance,
    rattle=0.1,
    scale=1.05,
    shuffle=True,
    seed=1,
    log=True,
)

write("mixture.xyz", atoms)

print(atoms)

atoms.calc = LennardJones(sigma=1.5 * tolerance, epsilon=1.0, rc=None, smooth=True)
optimizer = FIRE(atoms)
optimizer.run(fmax=1)

write("relaxed.xyz", atoms)


vpi = VaspInteractive(
    prec="Accurate",
    xc="pbe",
)
# Open a socket on default port localhost:31415

with SocketIOCalculator(vpi) as calc:
    npt = NPT(
        atoms,
        timestep=1.0 * units.fs,
        temperature_K=1100,
        externalstress=0,
        ttime=25.0 * units.fs,
        pfactor=(75 * units.fs) ** 2 * units.GPa,
        mask=np.eye(3)
    )
    atoms.calc = calc
    npt.run(steps=int(1e1))

Retrieving ThermoDoc documents:   0%|          | 0/7 [00:00<?, ?it/s]

8 NaCl 1.3333333333333333 Na6Cl6
16.0 NaCl 2.6666666666666665 Na6Cl6
24.0 NaCl 4.0 Na6Cl6


Retrieving ThermoDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

30.0 UCl3 15.0 U2Cl6
[{'name': 'Na6Cl6', 'number': 4, 'coords': Molecule Summary
Site: Na (2.2752, 3.9407, 3.5474)
Site: Na (2.2493, 0.0000, 3.5474)
Site: Na (2.2752, -3.9407, 3.5474)
Site: Na (1.1443, 1.9820, 0.0000)
Site: Na (4.5111, 0.0000, 0.0000)
Site: Na (1.1443, -1.9820, 0.0000)
Site: Cl (3.3998, -1.9629, 1.7769)
Site: Cl (3.3998, -1.9629, 5.3180)
Site: Cl (3.3998, 1.9629, 5.3180)
Site: Cl (3.3998, 1.9629, 1.7769)
Site: Cl (0.0000, 0.0000, 5.3276)
Site: Cl (0.0000, 0.0000, 1.7672), 'volume': 284.08233753792797}, {'name': 'U2Cl6', 'number': 15, 'coords': Molecule Summary
Site: U (3.7588, 2.1701, 1.0639)
Site: U (3.7588, -2.1701, 3.1916)
Site: Cl (2.9343, -3.9770, 1.0639)
Site: Cl (4.9113, 0.5527, 3.1916)
Site: Cl (4.5832, 3.9770, 3.1916)
Site: Cl (2.6062, -0.5527, 1.0639)
Site: Cl (5.7358, -1.9807, 1.0639)
Site: Cl (1.7818, 1.9807, 3.1916), 'volume': 208.27211476689524}]
Atoms(symbols='Cl114Na24U30', pbc=True, cell=[17.021838562107188, 17.021838562107188, 17.021838562107188])
   

RuntimeError: Looking for PP for potpaw_PBE/Cl/POTCAR
                        The pseudopotentials are expected to be in:
                        LDA:  $VASP_PP_PATH/potpaw/
                        PBE:  $VASP_PP_PATH/potpaw_PBE/
                        PW91: $VASP_PP_PATH/potpaw_GGA/
                        
                        No pseudopotential for Cl!